In [ ]:
import os, sys,pytz
import django
from datetime import datetime
sys.path.append('../..') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps

django.setup()

Remove mother from the c_sec_quart1_schedule1 schedule and put her in again on c_sec_quart1_schedule1 schedule,
Remove mother from the child_c_sec_qt_schedule1 schedule and put her in again on child_c_sec_qt_schedule1 schedule,
which corrects both mother and child schedules ,then link the mothers schdule to the child


And remove the already done appointments


In [ ]:
from edc_visit_schedule.models import SubjectScheduleHistory

subject_identifier = 'B142-040990025-4'
child_subject_identifier ='B142-040990025-4-10'
schedule_name_mother = 'c_sec_quart1_schedule1'
schedule_name_child = 'child_c_sec_qt_schedule1'

subject_history_obj = SubjectScheduleHistory.objects.get(
subject_identifier=subject_identifier,
schedule_name = schedule_name_mother
)

subject_history_obj_child = SubjectScheduleHistory.objects.get(
subject_identifier=child_subject_identifier,
schedule_name = schedule_name_child
)

onschedule_model_cls= django_apps.get_model(subject_history_obj.onschedule_model)
onschedule_model_cls_child= django_apps.get_model(subject_history_obj_child.onschedule_model)

onschedule_model_obj_child = onschedule_model_cls_child.objects.get(
    subject_identifier=child_subject_identifier,
    schedule_name=schedule_name_child,
    onschedule_datetime=subject_history_obj_child.onschedule_datetime,
)
onschedule_model_obj = onschedule_model_cls.objects.get(
    subject_identifier=subject_identifier,
    schedule_name=schedule_name_mother,
    onschedule_datetime=subject_history_obj.onschedule_datetime,
)
onschedule_datetime = onschedule_model_obj_child.onschedule_datetime
onschedule_model_obj.delete()
onschedule_model_obj_child.delete()
subject_history_obj.delete()
subject_history_obj_child.delete()

In [ ]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from flourish_caregiver.models import MaternalVisit
tz = pytz.timezone('Africa/Gaborone')

maternal_visit = MaternalVisit.objects.get(subject_identifier=subject_identifier,visit_code='2000M')
report_datetime = maternal_visit.report_datetime
report_datetime = report_datetime.astimezone(tz)

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_caregiver.onschedulecohortcsecquart',name=schedule_name_mother
)
schedule.put_on_schedule(
    subject_identifier=subject_identifier,onschedule_datetime=onschedule_datetime,
    schedule_name=schedule_name_mother,base_appt_datetime=report_datetime
)
print("Schedule created for",schedule)

In [ ]:
from edc_visit_schedule.site_visit_schedules import site_visit_schedules

_, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
    onschedule_model='flourish_child.onschedulechildcohortcsecquart',name=schedule_name_child
)
schedule.put_on_schedule(
    subject_identifier=child_subject_identifier,onschedule_datetime=onschedule_datetime,
    schedule_name=schedule_name_child,base_appt_datetime=report_datetime
)
print("Schedule created for",schedule)


In [ ]:
from flourish_child.models.onschedule import OnScheduleChildCohortCSecQuart
from flourish_caregiver.models import OnScheduleCohortCSecQuart
child_schedule = OnScheduleChildCohortCSecQuart.objects.get(subject_identifier=child_subject_identifier)


onschdule_mother= OnScheduleCohortCSecQuart.objects.get(subject_identifier=subject_identifier)
onschdule_mother.child_subject_identifier =child_schedule.subject_identifier
onschdule_mother.save()

print("Linked with child")

In [ ]:
from edc_appointment.models import Appointment

appointments_visit_codes = ['2013M','2014M','2015M','2016M']
mother_appointments = Appointment.objects.filter(subject_identifier=subject_identifier,
                                                 schedule_name=schedule_name_mother)

for appointemnt in mother_appointments:
    if appointemnt.visit_code not in appointments_visit_codes:
        print(f'Deleting {appointemnt.visit_code}')
        appointemnt.delete()

In [ ]:
from flourish_child.models import Appointment 

appointments_visit_codes = ['2013','2014','2015','2016']
child_appointments = Appointment.objects.filter(subject_identifier=child_subject_identifier,
                                                 schedule_name=schedule_name_child)

for appointemnt in child_appointments:
    if appointemnt.visit_code not in appointments_visit_codes:
        print(f'Deleting {appointemnt.visit_code}')
        appointemnt.delete()